In [13]:
%cd ./n9_rrs_rl/

[Errno 2] No such file or directory: './n9_rrs_rl/'
/home/cleversonahum/Documents/ai6g/n9_rrs_rl


# Simple scenario with 2 UEs and 2 basestations

The configuration files are stored in the `n9_rrs_rl/env_config/` folder. We can edit the config file in order to deploy the desired communication system scenario. The `n9_rrs_rl/env_config/simple.yml` file in the cell below defines a scenario with 2 basestations, where each basestation has a bandwidth of 2Hz and 2 resource blocks (RBs - Allocation unit). Therefore, each RB corresponds to 1Hz in each basestation.

In this simple scenario we consider only one slice, which can be assumed as a scenario without RAN slicing (since all the network belongs to the same slice).

We have 2 UEs in the system, where each UE has a maximum buffer latency of 10 steps (or milliseconds in case you consider each step as 1ms), so every time a packet waits more than 10ms in the buffer it should be dropped. The maximum number of packets in the buffer are 20 and 10 packets for UE 1 and 2, respectively. So every time UE 1 has a full buffer status (20 packets in the buffer), it should drop the newly arrived packets since it does not have sufficient buffer space to store it. We define the packet size for each UE as 1 bit.

The variable `basestation_ue_assoc` in the basestation config section indicates the UEs association for each basestation. It has a dimension $B$ x $U$, where $B$ is the total number of basestations and $U$ is the total number of UEs. The value 1 indicates that the UE is associated to the specified basestation and 0 otherwise. Therefore both UEs 1 and 2 are associated to basestations 1 and 2.

### Channel

Different channel implementation can be stored into `n9_rrs_rl/channels/` folder. A simple channel implementation returning constant spectral efficiency values is implemented in the `n9_rrs_rl/channels/simple.py` (code cell below) file to facilitate to understand our simple scenario and the simulator functions. The `SimpleChannel` inherits from the Channel class of sixg_radio_mgmt simulator, where the function step needs to return the spectral efficiency values for each BS-UE-RB, so the `spectral_efficiencies` variable has dimensions $B$ x $U$ x $R$, where $B$ is the total number of basestations, $U$ is the total number of UEs, and $R$ is the total number of RBs in each basestation. Therefore, `spectral_efficiencies[b,u,r]` represents the spectral efficiency (bit/Hz) in the basestation $b$, for UE $u$ in the RB $r$.

The `SimpleChannel` presented in the cell below (`n9_rrs_rl/channels/simple.py` file) always return a constant matrix with spectral efficiency values equals to 1 bit/Hz (`[ [[1, 1], [1, 1]], [[1, 1],[1, 1]] ]`). Therefore, every time a RB (1 Hz) is allocated to a UE, the UE will have a 1 bit/Hz throughput that corresponds to send one packet (since each packet has 1 bit). So each RB can transport one packet in our example.

It is important to emphasize that the `step()` function from `Channel` class receives an argument corresponding to the UEs mobility information, so the spectral efficiency calculation could use mobility information when implementing a realistic channel implementation.

In [14]:
# %load channels/simple.py
import numpy as np

from sixg_radio_mgmt import Channel


class SimpleChannel(Channel):
    def __init__(
        self,
        max_number_ues: int,
        max_number_basestations: int,
        num_available_rbs: np.ndarray,
        rng: np.random.Generator = np.random.default_rng(),
    ) -> None:
        super().__init__(
            max_number_ues, max_number_basestations, num_available_rbs, rng
        )

    def step(
        self, step_number: int, episode_number: int, mobilities: np.ndarray
    ) -> np.ndarray:
        spectral_efficiencies = [
            np.ones((self.max_number_ues, self.num_available_rbs[i]))
            for i in np.arange(self.max_number_basestations)
        ]

        return np.array(spectral_efficiencies)


### Mobility

Different mobility implementations can be stored into `n9_rrs_rl/mobilities/` folder. The Mobility files are responsible to generate the UEs movement and provide UE's localization, that could be used by the channel generation in case the Channel implementation uses it (as explained before). Note that in case you import channels from an external simulator, it already provides the spectral efficiencies for each UE in each RB, so it would not use the mobility information (since it already used it in the channel generation). Therefore, the mobility information will only be used in case you want to calculate the channel information online. Since we are considering that the `SimpleChannel` is returning constant spectral efficiency values, so there is no need to implement mobilities to the UEs, and we created a file `n9_rrs_rl/mobilities/simple.py` just to illustrate how it should work in case our scenario uses it.

Similar to the `Channel`class, the `Mobility` class `step()` function returns a matrix with dimensions $U$ x $2$ with constant values. Where each line represents the 2-D localization coordinate of each UE in the scenario.

In [15]:
# %load mobilities/simple.py
import numpy as np

from sixg_radio_mgmt import Mobility


class SimpleMobility(Mobility):
    def __init__(
        self, max_number_ues: int, rng: np.random.Generator = np.random.default_rng()
    ) -> None:
        super().__init__(max_number_ues, rng)

    def step(self, step_number: int, episode_number: int) -> np.ndarray:
        return np.ones((self.max_number_ues, 2))


### Traffic

The interpretation of the traffic flow in the simulator depends on the uplink or downlink assumption for the scenario. In case it assumes an uplink scenario, so the UEs generate data to be sent over the network to the basestation. In case it assumes a downlink scenario, the basestation generates the data to be sent over the network to the UEs. Let's assume an uplink scenario, so each UE generates data to be sent over the network to the basestation and its capacity to send information over the network depends on the number of RBs assigned to the UE and its spectral efficiency values.

The `Traffic` implementation into `n9_rrs_rl/traffics/` folder are responsible to define the amount of data that each UE would like to send over the network in each simulation step. It is important to emphasize that this amount of data depends on the assumption of application running in the UE, e.g. a UE running a video streaming application will have a different traffic pattern than a UE sending an email.

To facilitate our scenario interpretation, here we present a simple traffic scenario in `n9_rrs_rl/traffics/simple.py` file that return a constant number of bits per UE (traffic) as presented in the code cell below. The `step()` function needs to return a vector containing the traffic throughput requested by each UE in bits. In this `SimpleTraffic` implementation, we implemented that each UE will receive 4 bits per step to store in its buffer to be sent over the network after. Considering the scenario configuration, where each packet has a size of 1 bit, so each UE receives 4 packets per step.

In [16]:
# %load traffics/simple.py
import numpy as np

from sixg_radio_mgmt import Traffic


class SimpleTraffic(Traffic):
    def __init__(
        self, max_number_ues: int, rng: np.random.Generator = np.random.default_rng()
    ) -> None:
        super().__init__(max_number_ues, rng)

    def step(self, step_number: int, episode_number: int) -> np.ndarray:
        return np.ones(self.max_number_ues) * 4


### Agent

Now that our scenario is configured and all the mobility, traffic and channel functions are defined, we can start to model our agent that will be the responsible to allocate the radio resources (aka RBs) available in the basestations to the UEs. Our agent implementations are organized in the folder `n9_rrs_rl/agents/`. We implemented a simple round-robing agent that equally distributes the RBs available among the UEs in the agent `n9_rrs_rl/agents/round_robin.py` shown in the code cell below.

The `sixg_radio_mgmt` simulator provides an abstract class called `Agent` that shows all the required methods that the agent implementation should provide. In the case of our round-robin agent, we have the main function `step()` (similarly to the other classes) that is the responsible to allocate the RBs for each UE in each basestation. Let's start diving in the other auxiliary functions before diving into `step()` function.

The `obs_space_format()` function formats the observation space filtering all the information provided by the `sixg_radio_mgmt` environment in accordance to our agent needs. For example, the simulator environment in each step provide us with the following environment information:
- "pkt_incoming"
- "pkt_throughputs"
- "pkt_effective_thr"
- "buffer_occupancies"
- "buffer_latencies"
- "dropped_pkts"
- "mobility"
- "spectral_efficiencies"
- "basestation_ue_assoc"
- "basestation_slice_assoc"
- "slice_ue_assoc"
- "sched_decision"
- "slice_req"

But in the `obs_space_format()` function, we are selecting only the `basestation_ue_assoc` since it is the unique information needed for the round-robin implementation.

The `action_format()` is the responsible to format the agent `step()` function output (scheduling decision) to the format expected by the simulator. The scheduling decision is formatted as a matrix with dimensions $B$ x $U$ x $R$ with each element having value 1 to indicate that the RB was allocated to the UE in a specific basestation, or 0 otherwise. A possible scheduling decision variable for our scenario is `[ [[1, 0], [0, 1]], [[0, 1],[1, 0]] ]`, which indicates that in the first basestation the RB 1 was allocated to the UE 1, and the second RB was allocated to UE 2. In the basestation 2, the UE 1 received the RB 2 and the UE 2 received the RB 1.

Independent of the method used (RL, ML, heuristic...) the `action_format()` function needs to convert the method output to the format accepted by the simulator. In our scenario of the cell below, the `step()` function output already obbeys the matrix dimensions $B$ x $U$ x $R$, so the `action_format()` only returns the action generated by the `step()` function.

The `calculated_reward()` function is usually associated to the RL training process, so it is not needed in this moment that we are implementing a round-robin agent.

Finally, the `step()` function will receive the filtered observation space obtained from `obs_space_format()` function, and it will generate the scheduling decision to be used in the `action_format()` function to be used in the `sixg_radio_mgmt` environment. The `step()` function first creates an `allocation_rbs` variable with dimension $B$ x $U$ x $R$, and after starts to equally allocate the RBs available for each UE in the basestations that they are connected.

In [17]:
# %load agents/round_robin.py
from typing import Union

import numpy as np

from sixg_radio_mgmt import Agent, CommunicationEnv


class RoundRobin(Agent):
    def __init__(
        self,
        env: CommunicationEnv,
        max_number_ues: int,
        max_number_basestations: int,
        num_available_rbs: np.ndarray,
    ) -> None:
        super().__init__(
            env, max_number_ues, max_number_basestations, num_available_rbs
        )

    def step(self, obs_space: Union[np.ndarray, dict]) -> np.ndarray:
        allocation_rbs = [
            np.zeros((self.max_number_ues, self.num_available_rbs[basestation]))
            for basestation in np.arange(self.max_number_basestations)
        ]
        for basestation in np.arange(self.max_number_basestations):
            ue_idx = 0
            rb_idx = 0
            while rb_idx < self.num_available_rbs[basestation]:
                if obs_space[basestation][ue_idx] == 1:
                    allocation_rbs[basestation][ue_idx][rb_idx] += 1
                    rb_idx += 1
                ue_idx += 1 if ue_idx + 1 != self.max_number_ues else -ue_idx

        return np.array(allocation_rbs)

    def obs_space_format(self, obs_space: dict) -> np.ndarray:
        return np.array(obs_space["basestation_ue_assoc"])

    def calculate_reward(self, obs_space: dict) -> float:
        return 0

    def action_format(self, action: np.ndarray) -> np.ndarray:
        return action


### Executing the round-robin agent in the configured environment


In [18]:
import numpy as np
from tqdm import tqdm

from agents.round_robin import RoundRobin
from channels.simple import SimpleChannel
from mobilities.simple import SimpleMobility
from sixg_radio_mgmt import CommunicationEnv
from traffics.simple import SimpleTraffic

seed = 10
rng = np.random.default_rng(seed) if seed != -1 else np.random.default_rng()
comm_env = CommunicationEnv(
    SimpleChannel,
    SimpleTraffic,
    SimpleMobility,
    "simple",
    rng=rng,
)

round_robin = RoundRobin(comm_env, 2, 2, np.array([2, 2]))
comm_env.set_agent_functions(
    round_robin.obs_space_format,
    round_robin.action_format,
    round_robin.calculate_reward,
)

obs = comm_env.reset()
number_steps = 10
for step_number in tqdm(np.arange(comm_env.max_number_steps)):
    sched_decision = round_robin.step(obs)
    obs, _, end_ep, _ = comm_env.step(sched_decision)
    if end_ep:
        comm_env.reset()

100%|██████████| 10/10 [00:00<00:00, 327.05it/s]
